In [1]:
!nvidia-smi

Wed Apr 14 16:25:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.89       Driver Version: 465.89       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   62C    P8    17W /  N/A |    164MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";   

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
import pandas as pd

train_df = pd.read_csv('preparing_train_with_context5.csv').dropna(subset=['text_lemm']).reset_index(drop=True)
test_df = pd.read_csv('preparing_test_with_context5.csv').dropna(subset=['text_lemm']).reset_index(drop=True)

classes = train_df.topic.unique().tolist()

train_df['target'] = train_df.topic.map(lambda x: classes.index(x))
test_df['target'] = test_df.topic.map(lambda x: classes.index(x))

print('size of training set: %s' % (len(train_df)))
print('size of validation set: %s' % (len(test_df)))
print('classes: %s' % (classes))

size of training set: 416
size of validation set: 104
classes: ['Music', 'Art_Event', 'Movies_TV', 'Other', 'Sports', 'Games', 'SciTech', 'Phatic', 'Travel_Geo', 'Pets_Animals', 'Entertainment', 'News', 'Psychology', 'Politics', 'Celebrities', 'Religion', 'Literature', 'Fashion', 'Food_Drink', 'Weather_Time']


In [6]:
x_train = train_df['text_with_context'].values
y_train = train_df['target'].values
x_test = test_df['text_with_context'].values
y_test = test_df['target'].values

In [7]:
import ktrain
from ktrain import text

In [8]:
trn, val, preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                          x_test=x_test, y_test=y_test,
                                          class_names=classes,
                                          preprocess_mode='bert',
                                          maxlen=500)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [9]:
model = text.text_classifier('bert', train_data=trn, preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


In [10]:
learner = ktrain.get_learner(model, train_data=trn, batch_size=6)

In [11]:
learner.lr_find()
learner.lr_plot()

simulating training for different learning rates... this may take a few moments...
Epoch 1/1024
 1/70 [..............................] - ETA: 32:20 - loss: 3.3775 - accuracy: 0.0000e+00

ValueError: Please call lr_find first.

In [12]:
learner.autofit(2e-5, 5)



begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/5
70/70 [==============================] - 627s 9s/step - loss: 2.7672 - accuracy: 0.1731
Epoch 2/5
51/70 [====================>.........] - ETA: 2:49 - loss: 2.2423 - accuracy: 0.3595

KeyboardInterrupt: 

In [ ]:
learner.validate(val_data=val)

              precision    recall  f1-score   support

           0       0.73      0.86      0.79        22
           1       0.70      0.35      0.47        20
           2       0.59      0.80      0.68        20
           3       0.61      0.69      0.65        16
           4       0.70      0.74      0.72        19
           5       0.72      0.54      0.62        24
           6       0.42      0.32      0.36        25
           7       1.00      0.77      0.87        22
           8       0.80      0.76      0.78        21
           9       0.81      0.72      0.76        29
          10       0.83      1.00      0.91        20
          11       0.68      0.88      0.77        26
          12       0.75      0.50      0.60        18
          13       0.62      0.81      0.70        26
          14       0.83      0.83      0.83        23
          15       1.00      0.67      0.80        18
          16       0.52      0.61      0.56        23
          17       0.60    

array([[19,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         1,  0,  0],
       [ 5,  7,  1,  0,  2,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  1],
       [ 0,  0, 16,  0,  1,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,
         0,  0,  1],
       [ 0,  0,  0, 11,  1,  1,  0,  0,  0,  0,  0,  1,  0,  0,  1,  0,
         0,  1,  0],
       [ 0,  0,  0,  2, 14,  1,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,
         0,  0,  0],
       [ 0,  0,  2,  1,  0, 13,  1,  0,  0,  0,  0,  4,  0,  0,  1,  0,
         1,  1,  0],
       [ 0,  0,  1,  0,  0,  1,  8,  0,  0,  0,  0,  1,  0,  2,  0,  0,
         5,  0,  7],
       [ 0,  0,  0,  0,  0,  0,  2, 17,  0,  0,  1,  0,  0,  0,  0,  0,
         2,  0,  0],
       [ 2,  0,  1,  2,  0,  0,  0,  0, 16,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0],
       [ 0,  0,  3,  0,  0,  0,  1,  0,  0, 21,  0,  1,  1,  0,  0,  0,
         0,  1,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 20,  0,  0,  0,  0,  0

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
import pandas as pd
import dill

In [ ]:
predictor = ktrain.load_predictor('/content/drive/MyDrive/BERT/predictor_bert 27.03.json/')

In [ ]:
classes = predictor.get_classes()

In [ ]:
def get_subjects_by_general(df):
    subjects_obj = {}
    global_subj = ""

    for index, subj in subjects.iterrows():
        name = subj['Description']
        code = subj['ASJC Code']
        
        if code != code:
            global_subj = name
            continue

        subjects_obj[int(code)] = global_subj
    
    return subjects_obj

subjects = pd.read_excel('/content/drive/MyDrive/BERT/subjects.xlsx')
subjects_obj = get_subjects_by_general(subjects)

In [ ]:
df_preparing = pd.read_csv('/content/drive/MyDrive/BERT/preparing_motivation.csv')

In [ ]:
with open('/content/drive/MyDrive/BERT/Мотивация 2020.dictionary', 'rb') as file:
  df_motivation = pd.DataFrame(dill.load(file))

In [ ]:
df_motivation['text_lemm'] = df_motivation.eid.map(df_preparing.set_index('eid')['text_lemm'])

In [ ]:
df_motivation['subject'] = [[sub.area for sub in subjects] for subjects in df_motivation.subject_areas.to_list()]
df_motivation['subject_code'] = [[sub.code for sub in subjects] for subjects in df_motivation.subject_areas.to_list()]
df_motivation['general_subj'] = [[subjects_obj[int(sub)] for sub in subjects] for subjects in df_motivation.subject_code.to_list()]
# df_motivation['target'] = df_motivation.general_subj.map(lambda x: classes.index(x))

In [ ]:
df_motivation.dropna(subset=['text_lemm'], inplace=True)
df_motivation.reset_index(drop=True, inplace=True)

In [ ]:
import string
df_motivation['subject_clean'] = df_motivation['subject'].map(lambda x: "".join([w for w in " ".join(x).lower() if w not in string.punctuation]))

In [ ]:
df_motivation['keywords_clean'] = df_motivation['authkeywords'].map(lambda x: "".join([w for w in " ".join(x).lower() if w not in string.punctuation]) if x is not None else "")
df_motivation['idxterms_clean'] = df_motivation['idxterms'].map(lambda x: "".join([w for w in " ".join(x).lower() if w not in string.punctuation]) if x is not None else "")

In [ ]:
y_pred = predictor.predict((df_motivation['title'] + " " + df_motivation['keywords_clean']).values)

In [ ]:
cnt_all = 269
cnt_pos = 0

for ind, doc in df_motivation.iterrows():
  pred = y_pred[ind]
  
  if pred in doc.general_subj:
    cnt_pos += 1

In [ ]:
cnt_pos / cnt_all

0.4312267657992565

In [ ]:
y_pred_values = [classes.index(item) for item in y_pred]

In [ ]:
df_motivation['target'] = df_motivation.general_subj.map(lambda x: classes.index(x))
y_true = df_motivation['target'].values

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred_values)

0.3333333333333333

In [ ]:
classes

['Earth and Planetary Sciences',
 'Dentistry',
 'Psychology',
 'Biochemistry, Genetics and Molecular Biology',
 'Arts and Humanities',
 'Environmental Science',
 'Energy',
 'Mathematics',
 'Immunology and Microbiology',
 'Engineering',
 'Computer Science',
 'Neuroscience',
 'Pharmacology, Toxicology and Pharmaceutics',
 'Chemical Engineering',
 'Social Sciences',
 'Chemistry',
 'Agricultural and Biological Sciences',
 'Veterinary',
 'Nursing',
 'Business, Management and Accounting',
 'Economics, Econometrics and Finance',
 'Multidisciplinary',
 'Physics and Astronomy',
 'Medicine',
 'Materials Science']

In [ ]:
for ind in range(len(y_pred)):
 print(y_pred[ind], '|||', df_motivation['general_subj'][ind])

Chemistry ||| ['Chemistry']
Computer Science ||| ['Physics and Astronomy']
Engineering ||| ['Materials Science', 'Physics and Astronomy', 'Engineering']
Physics and Astronomy ||| ['Physics and Astronomy']
Materials Science ||| ['Materials Science']
Computer Science ||| ['Chemistry', 'Physics and Astronomy', 'Physics and Astronomy', 'Physics and Astronomy', 'Materials Science']
Computer Science ||| ['Chemistry', 'Physics and Astronomy', 'Physics and Astronomy', 'Physics and Astronomy', 'Materials Science']
Biochemistry, Genetics and Molecular Biology ||| ['Multidisciplinary']
Agricultural and Biological Sciences ||| ['Agricultural and Biological Sciences', 'Agricultural and Biological Sciences']
Mathematics ||| ['Mathematics', 'Mathematics']
Mathematics ||| ['Mathematics', 'Mathematics']
Engineering ||| ['Engineering', 'Engineering', 'Materials Science']
Business, Management and Accounting ||| ['Engineering', 'Engineering', 'Engineering', 'Engineering', 'Engineering']
Engineering ||| ['

In [4]:
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', remove=remove)
newsgroups_test = fetch_20newsgroups(subset='test', remove=remove)
texts = newsgroups_train.data +  newsgroups_test.data

KeyboardInterrupt: 

In [5]:
newsgroups_train = fetch_20newsgroups(subset='train')

KeyboardInterrupt: 

In [3]:
from sklearn.datasets import fetch_20newsgroups